<a href="https://colab.research.google.com/github/henrywoo/MyML/blob/master/Copy_of_nlu_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2018 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word Embeddings by Prediction

Please **make a copy** of this Colab notebook before starting this lab. To do so, choose **File**->**Save a copy in Drive**.

## Lab Outline
  1. Continuous Bag-of-Words (CBOW)
  1. The model
  1. Parameters and implementation
  1. Constructing the model
  1. Sampled Softmax
  1. Batches
  1. Nearest neighbors
  1. Scoring
  1. Predicting center words
  1. Takeaways
  1. Experiments

In this notebook, we'll discuss the famous `word2vec` algorithm. In contrast to strongly supervised approaches like WordNet or distributional methods like Brown Clustering or SVD, `word2vec` learns vector representations as a by-product of a predictive task.

Consider the task of sentence completion (a form of [Cloze test](https://en.wikipedia.org/wiki/Cloze_test)):

`the quick brown _____ jumped over the lazy dog`

You've probably seen [this sentence](https://en.wikipedia.org/wiki/The_quick_brown_fox_jumps_over_the_lazy_dog) enough to know the answer is `fox`. At least you know enough about *jumping*, average sizes of *dogs*, the spacial relationship implied by the preposition *over*, and a reasonable idea of what things in the world can be described as both *quick* and *brown* to narrow this down quite a bit. We'd like our model to do the same -- that is, infer a reasonable probability distribution over the words in a vocabulary given this context. In designing this model, we add a bottleneck: words must be represented by fixed-length vectors (embeddings). In order to predict the missing word, our model will need to learn good vectors $\mathbf{v}_i$ that represent the meaning of this word, and the words around it.

We'll describe two closely related models for this task, the Continuous Bag of Words (CBOW) model and the Skip-Gram models (often known as SGNS). Collectively, there are known as `word2vec`, and were originally introduced by:

- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf) (Mikolov et al. 2013a)
- [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf) (Mikolov et al. 2013b)


We'll train these models using gradient descent -- specifically, **backpropagation** as is used to train neural networks. We start with a random set of embeddings, then slowly adjust them to make better predictions. This is similar to training any machine learning model, where  the embeddings are our parameters. As a result, we'll train the `word2vec` models using neural network machinery (Tensorflow) even though the models themselves have just a single layer (the embeddings), and no non-linearities (the neural part of neural networks).

### Additional Reading

Before (or after) we dig into the details of CBOW and Skip-Gram below, you may want to read more about word embeddings. In addition to the `word2vec` papers above, you may find the following helpful:

- [Deep Learning, NLP, and Representations](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/) (Chris Olah, 2014)
- [TensorFlow Projector](http://projector.tensorflow.org/) to visualize `word2vec` in your browser.
- [GloVe](https://nlp.stanford.edu/projects/glove/), another popular method of creating embeddings that bridges the gap between `word2vec` and SVD.


## Setup

Let's get started with the imports needed for the rest of this lab.

We'll also need to load some utility functions which will help us work with the corpus.

### Imports

Run this code cell to add in all of the imports.

In [0]:
from __future__ import division
import time
import nltk
import numpy as np
import tensorflow as tf

### bAbI task corpus reader

Start by running the cell below to load the corpus reader.

In [0]:
#@title bAbI Task corpus reader

import sys, os
import glob
import re
from collections import namedtuple

# Struct types for different lines in the bAbI dataset.
# StoryLine represents "ID text" lines as (int, string)
# QALine represents "ID question answer support" lines as
# (int, string, string, list(int)).
# If tokenized, string fields can be replaced with list(string).
StoryLine = namedtuple("StoryLine", ["id", "text"])
QALine = namedtuple("QALine", ["id", "question", "answer", "support_ids"])

class BabiTaskCorpusReader(object):
    """Corpus reader for the bAbI tasks dataset.

    See https://research.fb.com/downloads/babi/ for details.

    This class exposes a similar interface to NLTK's corpus readers, and should
    be interchangable with them in many applications.

    Example usage:

    import babi_utils
    import nltk
    tok = nltk.tokenize.treebank.TreebankWordTokenizer()
    cr = babi_utils.BabiTaskCorpusReader("/home/babi/en",
                                         tokenizer=tok.tokenize)
    words = list(cr.words())
    print words[:8]
    # ['John', 'travelled', 'to', 'the', 'hallway', '.', 'Mary', 'journeyed']

    """

    ALL_FILES = [
        'qa10_indefinite-knowledge_test.txt',
        'qa10_indefinite-knowledge_train.txt',
        'qa11_basic-coreference_test.txt',
        'qa11_basic-coreference_train.txt',
        'qa12_conjunction_test.txt',
        'qa12_conjunction_train.txt',
        'qa13_compound-coreference_test.txt',
        'qa13_compound-coreference_train.txt',
        'qa14_time-reasoning_test.txt',
        'qa14_time-reasoning_train.txt',
        'qa15_basic-deduction_test.txt',
        'qa15_basic-deduction_train.txt',
        'qa16_basic-induction_test.txt',
        'qa16_basic-induction_train.txt',
        'qa17_positional-reasoning_test.txt',
        'qa17_positional-reasoning_train.txt',
        'qa18_size-reasoning_test.txt',
        'qa18_size-reasoning_train.txt',
        'qa19_path-finding_test.txt',
        'qa19_path-finding_train.txt',
        'qa1_single-supporting-fact_test.txt',
        'qa1_single-supporting-fact_train.txt',
        'qa20_agents-motivations_test.txt',
        'qa20_agents-motivations_train.txt',
        'qa2_two-supporting-facts_test.txt',
        'qa2_two-supporting-facts_train.txt',
        'qa3_three-supporting-facts_test.txt',
        'qa3_three-supporting-facts_train.txt',
        'qa4_two-arg-relations_test.txt',
        'qa4_two-arg-relations_train.txt',
        'qa5_three-arg-relations_test.txt',
        'qa5_three-arg-relations_train.txt',
        'qa6_yes-no-questions_test.txt',
        'qa6_yes-no-questions_train.txt',
        'qa7_counting_test.txt',
        'qa7_counting_train.txt',
        'qa8_lists-sets_test.txt',
        'qa8_lists-sets_train.txt',
        'qa9_simple-negation_test.txt',
        'qa9_simple-negation_train.txt'
    ]

    def __init__(self, directory, mask="qa*.txt",
                 file_list=ALL_FILES,
                 file_reader=open,
                 tokenizer=lambda s: s.split(),
                 verbose=False):
        """Construct a corpus reader for the bAbI tasks dataset.

        Args:
            directory: (string) path to bAbI text files (e.g. /home/babi/en/)
            mask: (string) file glob to match particular files. Use
                "qa16_*" e.g. to match task 16.
            file_list: (list(string) or None) If None, will glob directory to
                find files. Otherwise, will use the given list of basenames.
            file_reader: (function string -> fd) optional replacement for
                Python's built-in open(...) method, to be used for reading
                from alternative file-like objects.
            tokenizer: function string -> list(string), used to split
                sentences.
            verbose: (bool) if true, will print when reading files.
        """
        self._open = file_reader
        self._tokenizer = tokenizer
        self._verbose = verbose

        if file_list:
            basenames = glob.fnmatch.filter(file_list, mask)
            filenames = [os.path.join(directory, f) for f in basenames]
        else:
            # Glob directory
            pattern = os.path.join(directory, mask)
            filenames = glob.glob(pattern)

        # Filenames of form qaXX_task-name_train.txt
        # Want to sort by XX as a number
        key_fn = lambda f: (int(os.path.basename(f).split("_")[0][2:]), f)
        self._filenames = sorted(filenames, key=key_fn)
        # Filenames should be nonempty!
        assert(self._filenames), "No files found matching [{:s}]".format(mask)

    def filenames(self):
        return self._filenames

    def parse_line(self, line):
        """Parse a single line from the bAbI corpus.

        Line is of one of the two forms:
        ID text
        ID question[tab]answer[tab]supporting fact IDs

        See https://research.fb.com/downloads/babi/

        Args:
            line: (string)

        Returns:
            (id, text) as (int, string)
            OR (id, question, answer, [ids]) as (int, string, string, list(int))
        """
        id_text, rest = line.split(" ", 1)
        id = int(id_text)
        if "\t" in rest:
            question, answer, s_ids_text = rest.split("\t")
            s_ids = map(int, s_ids_text.split())
            return QALine(id, question.strip(), answer.strip(), s_ids)
        else:
            return StoryLine(id, rest.strip())

    def tokenize_parsed_line(self, line):
        if isinstance(line, StoryLine):
            return StoryLine(line.id, self._tokenizer(line.text))
        else:
            return QALine(line.id,
                          self._tokenizer(line.question),
                          self._tokenizer(line.answer),
                          line.support_ids)

    def _line_iterator(self):
        for f in self._filenames:
            if self._verbose:
                print >> sys.stderr, "Reading {:s}".format(os.path.basename(f)),
            with self._open(f) as fd:
                for line in fd:
                    yield line.strip()
            if self._verbose:
                print >> sys.stderr, "...done!"

    def examples(self, tokenize=True):
        """Iterator over complete stories (training examples).

        A story spans multiple lines, of the form:

        1 text one
        2 text two
        3 text three
        4 question[tab]answer[tab]supporting fact IDs

        Args:
            tokenize: (bool) If true, will tokenize text fields.

        Returns:
            iterator yielding list(StoryLine|QALine)
              if tokenize=True, then text, question, and answer will be
              list(string); otherwise they will be plain strings.
        """
        buffer = []
        for line in self._line_iterator():
            parsed = self.parse_line(line)
            if tokenize:
                parsed = self.tokenize_parsed_line(parsed)
            # If new story item, flush buffer.
            if buffer and parsed.id <= buffer[-1].id:
                yield buffer
                buffer = []
            buffer.append(parsed)
        # Flush at end.
        yield buffer
        buffer = []

    def _raw_sents_impl(self, stories=False, questions=False, answers=False):
        for line in self._line_iterator():
            parsed = self.parse_line(line)
            if isinstance(parsed, StoryLine) and stories:
                yield parsed.text
            else:
                if questions:
                    yield parsed.question
                if answers:
                    yield parsed.answer

    def raw_sents(self):
        """Iterator over utterances in the corpus.

        Returns untokenized sentences.

        Returns:
            iterator yielding string
        """
        return self._raw_sents_impl(stories=True,
                                    questions=True,
                                    answers=True)

    def sents(self):
        """Iterator over utterances in the corpus.

        Returns tokenized sentences, a la NLTK.

        Returns:
            iterator yielding list(string)
        """
        for sentence in self.raw_sents():
            yield self._tokenizer(sentence)


    def words(self):
        """Iterator over words in the corpus.

        Returns:
            iterator yielding string
        """
        for sentence in self.sents():
            for word in sentence:
                yield word

### Corpus utilities

Next, let's load some utility functions which will help us work with the corpus.

In [0]:
#@title Vocabulary helper functions

import collections
from collections import defaultdict

class Vocabulary(object):

  START_TOKEN = u"<s>"
  END_TOKEN   = u"</s>"
  UNK_TOKEN   = u"<unk>"

  def __init__(self, tokens, size=None):
    """Create a Vocabulary object.

    Args:
        tokens: iterator( string )
        size: None for unlimited, or int > 0 for a fixed-size vocab.
              Vocabulary size includes special tokens <s>, </s>, and <unk>
    """
    self.unigram_counts = collections.Counter(tokens)
    self.bigram_counts = defaultdict(lambda: defaultdict(lambda: 0))
    word1 = None
    for word in tokens:
        if word1 is None:
            pass
        self.bigram_counts[word1][word] += 1
        word1 = word
    self.bigram_counts.default_factory = None  # make into a normal dict

    # Leave space for "<s>", "</s>", and "<unk>"
    top_counts = self.unigram_counts.most_common(None if size is None else (size - 3))
    vocab = ([self.START_TOKEN, self.END_TOKEN, self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in self.id_to_word.iteritems()}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience
    self.wordset = set(self.word_to_id.iterkeys())

    # Store special IDs
    self.START_ID = self.word_to_id[self.START_TOKEN]
    self.END_ID = self.word_to_id[self.END_TOKEN]
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]

  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def pad_sentence(self, words, use_eos=True):
    ret = [self.START_TOKEN] + words
    if use_eos:
      ret.append(self.END_TOKEN)
    return ret

  def sentence_to_ids(self, words, use_eos=True):
    return self.words_to_ids(self.pad_sentence(words, use_eos))

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))

In [0]:
#@title Utilities

import re
import time
import itertools
import numpy as np

# For pretty-printing
import pandas as pd
from IPython.display import display, HTML

UNK_TOKEN   = u"<unk>"

def flatten(list_of_lists):
    """Flatten a list-of-lists into a single list."""
    return list(itertools.chain.from_iterable(list_of_lists))

def pretty_print_matrix(M, rows=None, cols=None, dtype=float, float_fmt="{0:.04f}"):
    """Pretty-print a matrix using Pandas.

    Args:
      M : 2D numpy array
      rows : list of row labels
      cols : list of column labels
      dtype : data type (float or int)
      float_fmt : format specifier for floats
    """
    df = pd.DataFrame(M, index=rows, columns=cols, dtype=dtype)
    old_fmt_fn = pd.get_option('float_format')
    pd.set_option('float_format', lambda f: float_fmt.format(f))
    display(df)
    pd.set_option('float_format', old_fmt_fn)  # reset Pandas formatting

def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)


##
# Word processing functions
def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

##
# Data loading functions
def get_corpus(name="brown"):
    import nltk
    assert(nltk.download(name))
    return nltk.corpus.__getattr__(name)

def build_vocab(corpus, V=10000):
    token_feed = (canonicalize_word(w) for w in corpus.words())
    vocab = Vocabulary(token_feed, size=V)
    return vocab

def get_train_test_sents(corpus, split=0.8, shuffle=True):
    """Generate train/test split for unsupervised tasks.

    Args:
      corpus: nltk.corpus that supports sents() function
      split (double): fraction to use as training set
      shuffle (int or bool): seed for shuffle of input data, or False to just
      take the training data as the first xx% contiguously.

    Returns:
      train_sentences, test_sentences ( list(list(string)) ): the train and test
      splits
    """
    sentences = np.array(list(corpus.sents()), dtype=object)
    fmt = (len(sentences), sum(map(len, sentences)))
    print "Loaded {:,} sentences ({:g} tokens)".format(*fmt)

    if shuffle:
        rng = np.random.RandomState(shuffle)
        rng.shuffle(sentences)  # in-place
    train_frac = 0.8
    split_idx = int(train_frac * len(sentences))
    train_sentences = sentences[:split_idx]
    test_sentences = sentences[split_idx:]

    fmt = (len(train_sentences), sum(map(len, train_sentences)))
    print "Training set: {:,} sentences ({:,} tokens)".format(*fmt)
    fmt = (len(test_sentences), sum(map(len, test_sentences)))
    print "Test set: {:,} sentences ({:,} tokens)".format(*fmt)

    return train_sentences, test_sentences

def preprocess_sentences(sentences, vocab, use_eos=False, emit_ids=True):
    """Preprocess sentences by canonicalizing and mapping to ids.

    Args:
      sentences ( list(list(string)) ): input sentences
      vocab: Vocabulary object, already initialized
      use_eos: if true, will add </s> token to end of sentence.
      emit_ids: if true, will emit as ids. Otherwise, will be preprocessed
          tokens.

    Returns:
      ids ( array(int) ): flattened array of sentences, including boundary <s>
      tokens.
    """
    # Add sentence boundaries, canonicalize, and handle unknowns
    word_preproc = lambda w: canonicalize_word(w, wordset=vocab.word_to_id)
    ret = []
    for s in sentences:
        canonical_words = vocab.pad_sentence(map(word_preproc, s),
                                             use_eos=use_eos)
        ret.extend(vocab.words_to_ids(canonical_words) if emit_ids else
                   canonical_words)
    if not use_eos:  # add additional <s> to end if needed
        ret.append(vocab.START_ID if emit_ids else vocab.START_TOKEN)
    return np.array(ret, dtype=(np.int32 if emit_ids else object))


def load_corpus(corpus, split=0.8, V=10000, shuffle=0):
    """Load a named corpus and split train/test along sentences.

    This is a convenience wrapper to chain together several functions from this
    module, and produce a train/test split suitable for input to most models.

    Sentences are preprocessed by canonicalization and converted to ids
    according to the constructed vocabulary, and interspersed with <s> tokens
    to denote sentence bounaries.

    Args:
        corpus: (string | corpus reader) If a string, will fetch the
            NLTK corpus of that name.
        split: (float \in (0,1]) fraction of examples in train split
        V: (int) vocabulary size (including special tokens)
        shuffle: (int) if > 0, use as random seed to shuffle sentence prior to
            split. Can change this to get different splits.

    Returns:
        (vocab, train_ids, test_ids)
        vocab: vocabulary.Vocabulary object
        train_ids: flat (1D) np.array(int) of ids
        test_ids: flat (1D) np.array(int) of ids
    """
    if isinstance(corpus, str):
        corpus = get_corpus(corpus)
    vocab = build_vocab(corpus, V)
    train_sentences, test_sentences = get_train_test_sents(corpus, split, shuffle)
    train_ids = preprocess_sentences(train_sentences, vocab)
    test_ids = preprocess_sentences(test_sentences, vocab)
    return vocab, train_ids, test_ids

##
# Window and batch functions
def rnnlm_batch_generator(ids, batch_size, max_time):
    """Convert ids to data-matrix form for RNN language modeling."""
    # Clip to multiple of max_time for convenience
    clip_len = ((len(ids)-1) / batch_size) * batch_size
    input_w = ids[:clip_len]     # current word
    target_y = ids[1:clip_len+1]  # next word
    # Reshape so we can select columns
    input_w = input_w.reshape([batch_size,-1])
    target_y = target_y.reshape([batch_size,-1])

    # Yield batches
    for i in xrange(0, input_w.shape[1], max_time):
        yield input_w[:,i:i+max_time], target_y[:,i:i+max_time]


def build_windows(ids, N, shuffle=True):
    """Build window input to the window model.

    Takes a sequence of ids, and returns a data matrix where each row
    is a window and target for the window model. For N=3:
        windows[i] = [w_3, w_2, w_1, w_0]

    For language modeling, N is the context size and you can use y = windows[:,-1]
    as the target words and x = windows[:,:-1] as the contexts.

    For CBOW, N is the window size and you can use y = windows[:,N/2] as the target words
    and x = np.hstack([windows[:,:N/2], windows[:,:N/2+1]]) as the contexts.

    For skip-gram, you can use x = windows[:,N/2] as the input words and y = windows[:,i]
    where i != N/2 as the target words.

    Args:
      ids: np.array(int32) of input ids
      shuffle: if true, will randomly shuffle the rows

    Returns:
      windows: np.array(int32) of shape [len(ids)-N, N+1]
        i.e. each row is a window, of length N+1
    """
    windows = np.zeros((len(ids)-N, N+1), dtype=int)
    for i in xrange(N+1):
        # First column: first word, etc.
        windows[:,i] = ids[i:len(ids)-(N-i)]
    if shuffle:
        # Shuffle rows
        np.random.shuffle(windows)
    return windows


def batch_generator(data, batch_size):
    """Generate minibatches from data.

    Args:
      data: array-like, supporting slicing along first dimension
      batch_size: int, batch size

    Yields:
      minibatches of maximum size batch_size
    """
    for i in xrange(0, len(data), batch_size):
        yield data[i:i+batch_size]

In [0]:
#@title TSV Corpus Reader

import sys, os

class TSVCorpusReader(object):
    """Corpus reader for TSV files.

    Input files are assumed to contain one sentence per line, with tokens
    separated by tabs:

    foo[tab]bar[tab]baz
    span[tab]eggs

    Would correspond to the two-sentence corpus:
        ["foo", "bar", "baz"],
        ["spam", "eggs"]

    """

    def __init__(self, sentence_file, preload=True, file_reader=open):
        """Construct a corpus reader for the given file.

        Args:
            sentence_file: (string) path to a TSV file with one sentence per
                line.
            preload: (bool) If true, will read entire corpus to memory on
                construction. Otherwise, will load on-demand.
            file_reader: (function string -> fd) optional replacement for
                Python's built-in open(...) method, to be used for reading
                from alternative file-like objects.
        """
        self._open = file_reader
        self._sentence_file = sentence_file
        self._sentence_cache = []

        if preload:
            self._sentence_cache = list(self.sents())

    def _line_iterator(self):
        with self._open(self._sentence_file) as fd:
            for line in fd:
                yield line.strip()

    def sents(self):
        """Iterator over sentences in the corpus.

        Yields:
            list(string) of tokens
        """
        if self._sentence_cache:
            for sentence in self._sentence_cache:
                yield sentence
        else:
            # If no cache, actually read the file.
            for line in self._line_iterator():
                yield line.split("\t")

    def words(self):
        """Iterator over words in the corpus.

        Yields:
            (string) tokens
        """
        for sentence in self.sents():
            for word in sentence:
                yield word

### Loading the Corpus

We are going to be training on the [bAbI Tasks corpus](https://research.fb.com/downloads/babi/). This is an artificial corpus of around 1M words, which has the advantage of looking like natural language while being free from noise and having a very small vocabulary. This means we'll see each word many times, and so will have plenty of signal to get useful embeddings.

We'll pre-process the inputs, lowercasing and canonicalizing digits and adding `<s>` and `</s>` markers to sentence boundaries. This is done internally by the `load_corpus` function provided with utilities - for now, don't worry too much about how it works. The form we'll deal with is a list of ids: `train_ids` for the training set, and `test_ids` for a held-out development set.

In [0]:
!wget https://storage.googleapis.com/mledu-datasets/babi_tasks_1-20_v1-2.tar.gz -O /tmp/babi_tasks_1-20_v1-2.tar.gz

In [0]:
import os
import tarfile

local_tar = '/tmp/babi_tasks_1-20_v1-2.tar.gz'
tar_ref = tarfile.open(local_tar, 'r:gz')
tar_ref.extractall('/tmp')
tar_ref.close()

Note: running the following cell to load the corpus and split into training and test sets may take 20-30 seconds.

In [0]:
print 'Loading bAbI corpus... ',
babi_corpus = BabiTaskCorpusReader(
    "/tmp/tasks_1-20_v1-2/en",
    tokenizer=nltk.tokenize.treebank.TreebankWordTokenizer().tokenize)
print 'Done'

# Load dataset and split into train and test.
corpus = babi_corpus 
vocab, train_ids, test_ids = load_corpus(
    corpus, split=0.8, V=10000, shuffle=42)

## Continuous Bag-of-Words (CBOW)

The goal here is to solve the fill-in-the-blank task directly. We treat it as a supervised learning task, where the inputs are the surrounding words (the **context**), and the **target** is the center word:

$$ \begin{eqnarray}
x & = & \{\texttt{quick},\ \texttt{brown},\ \texttt{jumped},\ \texttt{over}\} \\
y & = & \ \texttt{fox}
\end{eqnarray} $$

We're interested in real-valued vectors, so let's pick a dimension $d$ (typically, $d = 100$ or so) and assign each word $i \in V$ in our vocabulary to a vector $\mathbf{w}_i \in \mathbb{R}^d$. Now we have embedded inputs, which look like:

$$ \begin{eqnarray}
\mathbf{x} & = & \{\mathbf{w}_\texttt{quick},
                 \ \mathbf{w}_\texttt{brown},
                 \ \mathbf{w}_\texttt{jumped},
                 \ \mathbf{w}_\texttt{over}\} \\
\mathbf{y} & = & \ \mathbf{u}_\texttt{fox}
\end{eqnarray} $$

and our goal is to predict $\mathbf{y}$ given $\mathbf{x}$. Note that we gave the target word a vector $\mathbf{u}_i$ denoted by a different letter - we'll see why this is below.

We can think of these embedding vectors a bit like feature vectors. In a classical modeling framework, we might have inputs $\mathbf{x}_i \in \mathbb{R}^k$, where each vector dimension $j = 1,\ldots,k$ represents a feature and we learn some parameters $\mathbf{\theta} \in \mathbb{R}^k$ to fit them. The key difference with `word2vec` is that instead, the features $\mathbf{w}_i$ won't be fixed -- their values are what we are trying to learn.

## The model

What is the actual form of this model, of our prediction function?

Let's take one step back, and let our input $x \subseteq V$ be a set of indices representing words. _(As with SVD embeddings, the actual indices are arbitrary - they need only be unique and consistent.)_ 

So $ x = \{\texttt{quick},\ \texttt{brown},\ \texttt{jumped},\ \texttt{over}\}$ might be represented as `{7, 42, 3, 127}`. We'll embed these by taking the vector representations (remember, these are model parameters and not fixed yet!), to get:

$$\mathbf{x} = \{\mathbf{w}_\texttt{quick},
               \ \mathbf{w}_\texttt{brown},
               \ \mathbf{w}_\texttt{jumped},
               \ \mathbf{w}_\texttt{over}\} 
             = \{\mathbf{w}_{7},
               \ \mathbf{w}_{42},
               \ \mathbf{w}_{3},
               \ \mathbf{w}_{127}\} 
$$

Now we'll take these embeddings and add them together to get a single vector $\mathbf{\bar{w}} \in \mathbb{R}^d$:

$$ \mathbf{\bar{w}} = \sum_{i \in x} \mathbf{w}_i = \mathbf{w}_{7} + \mathbf{w}_{42} + \mathbf{w}_{3} + \mathbf{w}_{127} $$

This is the origin of the name: "bag-of-words" because we're just adding the word representations together while ignoring position and order, and "continuous" because we're doing this on real-valued (continuous) vectors.

And now we'll take the dot product of this with vectors for _each possible word_ $j \in V$, add a bias term, and feed the result through a softmax to predict a probability that each word $j$ is the center word:

$$ \begin{eqnarray}
a_j & = & \mathbf{\bar{w}}\cdot \mathbf{u}_j + b_j \\
\mathbf{\hat{y}} & = & \text{softmax}(\mathbf{a})
\end{eqnarray} $$

where $\mathbf{a} \in \mathbb{R}^V$ are logits and $\mathbf{\hat{y}} \in [0,1]^V$ is a vector of our predicted probabilities ($\sum_{j} \hat{y}_j = 1$).

This last step should be very familiar - it's just logistic regression! Specifically, if we think of $\mathbf{\bar{w}}$ as a fixed set of features, then the CBOW model is just multiclass logistic regression with $V$ classes, one for each word in the vocabulary. The learned parameters are the "output vectors" $\mathbf{u}_j$, a weight for each feature, for each class.

## Parameters and implementation

Recall that our goal is to learn the word vectors $\mathbf{w}_i$ (and $\mathbf{u}_i$). For convenience, let's stack them into two matrices, which we'll call $W$ and $U$:

$$ W = \begin{pmatrix} 
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{w}_1\ \vcenter{\Rule{15px}{0.5px}{1px}}\\ 
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{w}_2\ \vcenter{\Rule{15px}{0.5px}{1px}}\\ 
\vdots \\
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{w}_V\ \vcenter{\Rule{15px}{0.5px}{1px}} 
\end{pmatrix} \ \in \mathbb{R}^{V \times d} 
\quad \text{and} \quad
U = \begin{pmatrix} 
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{u}_1\ \vcenter{\Rule{15px}{0.5px}{1px}}\\ 
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{u}_2\ \vcenter{\Rule{15px}{0.5px}{1px}}\\ 
\vdots \\
\vcenter{\Rule{15px}{0.5px}{1px}}\ \mathbf{u}_V\ \vcenter{\Rule{15px}{0.5px}{1px}} 
\end{pmatrix} \ \in \mathbb{R}^{V \times d} 
$$

We'll refer to $W$ as the input matrix (and its rows the input embeddings), and $U$ as the output matrix (and its rows the output embeddings). We'll also have one additional parameter, the biases $\mathbf{b} \in \mathbb{R}^V$.

_**Note:** Keeping two separate sets of vectors makes the model easier to train, but there isn't a major distinction otherwise - commonly, we can just concatenate them when we're done to get word embeddings $\mathbf{v}_i = [\mathbf{w}_i, \mathbf{u}_i]$._

Now we can write our model in matrix form, interpreting $x$ as a many-hot vector $\mathbf{x} \in \mathbb{R}^V$ where 
$$ x_i = \begin{cases}
1\ \text{if}\ i \in x \\
0\ \text{otherwise}
\end{cases} $$

So we have for our final model equations:
$$\begin{eqnarray}
\mathbf{\bar{w}} & = & W \mathbf{x} \\ 
\mathbf{a} & = & U \mathbf{\bar{w}} + \mathbf{b} \\
\mathbf{\hat{y}} & = & \text{softmax}(\mathbf{a})
\end{eqnarray}$$

And finally, we can write our loss function - the cross-entropy loss - exactly as in logistic regression:

$$ \mathcal{L}(W,U,\mathbf{b}) = \sum_{(x,\ y\ =\ j)\ \in\ \mathcal{D}} -\log \hat{y}_j(x) $$

Because we train both the input features $W$ and the class features $U$ at the same time, this problem is no longer convex. However, as with neural networks, we can still train it using gradient descent. In the cells below, we'll implement the above equations in TensorFlow, and use the powerful automatic differentiation features to set up training with just a few lines of code.

## Constructing the model

A TensorFlow implemenation of CBOW follows. We'll use the bAbI corpus as before, but encourage you to experiment with other text corpora.

To implement CBOW in TensorFlow, we need to define a Tensor for each model component. To clearly distinguish between TF and non-TF code, TF object names will be suffixed by an underscore ("_"). We also will construct the model such that it accept batch inputs to speed up training.

**Hyperparameters**
* V : vocabulary size
* M : embedding size
* N : context size (excluding the center word)

**Inputs**
* ids_ : (batch_size, N), integer indices for context words
* y_ : (batch_size,), integer indices for target word

**Model Parameters**
* V_ : (V,M), input-side word embeddings
* U_ : (V,M), output-side word embeddings
* b_ : (V,), biases for output-side

**Intermediate States**
* x_ : (batch_size, N, M), concatenated word embeddings of the context words
* z_ : (batch_size, M), average of context words' embeddings
* logits_ : (batch_size, V), $zU^T + b$

In [0]:
tf.reset_default_graph()
tf.set_random_seed(42)

# MODEL HYPERPARAMETERS
V = vocab.size
M = 25
N = 4

# INPUTS
# Using "None" in place of batch size allows it to be dynamically computed later.
with tf.name_scope("Inputs"):
    ids_ = tf.placeholder(tf.int32, shape=[None, N], name="ids")
    y_ = tf.placeholder(tf.int32, shape=[None], name="y")

# EMBEDDING LAYER COMPONENTS
# Here we grab the embedddings of the context words and sum them.
# All of the input embeddings are stored in V_.
with tf.name_scope("Embedding_Layer"):
    # Use random initialization for the input embeddings.
    V_ = tf.Variable(tf.random_uniform([V, M]), name="V")

    # We want the embedding_lookup to produce shape: (batch_size, N, M).
    # Passing [-1, N, M] to tf.reshape will fill in the batch_size dynamically.
    x_ = tf.reshape(tf.nn.embedding_lookup(V_, ids_), 
                    [-1, N, M], name="x")
    
    # We want to sum over the words in the window (N) for each example,
    # resulting in a shape for z_: (batch_size, M).
    z_ = tf.reduce_sum(x_, axis=1)

# OUTPUT LAYER COMPONENTS
with tf.name_scope("Output_Layer"):
    U_ = tf.Variable(tf.random_uniform([V,M]), name="U")
    b_ = tf.Variable(tf.zeros([V,], dtype=tf.float32), name="b")
    logits_ = tf.add(tf.matmul(z_, tf.transpose(U_)), b_, name="logits")

## Sampled Softmax

We'll add in our usual cross-entropy loss. The full **softmax** is computationally extremely slow because the denominator is a sum over the probabilities of every word in the vocabulary. To speed up training we'll use a clever trick called the **sampled softmax** loss, as in Jozefowicz et al. 2016. Sampled softmax approximates the denominator by replacing the exhaustive sum with a sum of a sample of random center words.

(Note: running the following code cell may give a warning that the softmax_cross_entropy_with_logits function is being deprecated and will be removed in a future version; you can ignore this warning for now.)

In [0]:
with tf.name_scope("Cost_Function"):
    # Sampled softmax loss, for training
    per_example_train_loss_ = tf.nn.sampled_softmax_loss(weights=U_, biases=b_,
                                             labels=tf.expand_dims(y_, 1), inputs=z_,
                                             num_sampled=100, num_classes=V,
                                             name="per_example_sampled_softmax_loss")
    train_loss_ = tf.reduce_mean(per_example_train_loss_, name="sampled_softmax_loss")
    
    # Full softmax loss, for scoring
    per_example_loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y_, logits=logits_, name="per_example_loss")
    loss_ = tf.reduce_mean(per_example_loss_, name="loss")

Let's add in our training ops. We will use AdaGrad instead of SGD; this is faster because it keeps track of how many times we have seen a particular example and modifying the learning rate accordingly. Why is this useful? Let's take the words "the" and "aardvark." We are likely going to see the word "the" many many more times than "aardvark". After seeing the word "the" a thousand times, we probably don't need to backpropagate the errors as much anymore because we've probably converged to the correct embedding. However, we don't see the word "aardvark" very often, so we'd like to keep the learning rate fairly high so that we can converge to its embedding faster. AdaGrad allows us to do this instead of keeping a single learning rate for all examples.

In [0]:
with tf.name_scope("Training"):
    alpha_ = tf.placeholder(tf.float32, name="learning_rate")
    optimizer_ = tf.train.AdagradOptimizer(alpha_)
    train_step_ = optimizer_.minimize(train_loss_)
    
# Initializer step
init_ = tf.global_variables_initializer()

Let's add a few more ops to do our final predictions after we have completed training:

* pred\_proba_ : (batch_size, V), $ = P(w_i | w_{i-1}, w_{i+1} ...)$ for all words $i$
* pred_max : (batch_size,), id of most likely middle word
* pred_random : (batch_size,), id of a randomly-sampled middle word

In [0]:
with tf.name_scope("Prediction"):
    pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
    pred_max_ = tf.argmax(logits_, 1, name="pred_max")
    pred_random_ = tf.multinomial(logits_, 1, name="pred_random")

## Batches

We will also need a function to generate batches, which will be used in the next section. Each batch consists of `batch_size` training examples, where each includes an array of context word ids, and a single id representing the middle (target) word.

In [0]:
# Create windows of words -- we'll use the middle word as
# the target, and the others as the input context.
train_windows = build_windows(train_ids, N)
test_windows = build_windows(test_ids, N)
print 'Training windows shape:', train_windows.shape
print 'Test windows shape:', test_windows.shape

In [0]:
def cbow_slice_batch(batch, N):
    """Extract center words as targets, left and right context as inputs."""
    # Get the center word from each example as the targets.
    targets = batch[:,N//2]  # The // operator does integer division
    
    # Get the context words as inputs.
    inputs = np.hstack([batch[:,:N//2], batch[:,(N//2+1):]])
    return inputs, targets

def train_batch(session, batch, alpha):
    inputs, targets = cbow_slice_batch(batch, N)
    # Prepare inputs for TensorFlow.
    feed_dict = {ids_:inputs, y_:targets, alpha_:alpha}
    c, _ = session.run([train_loss_, train_step_],
                       feed_dict=feed_dict)
    return c

## Nearest neighbors

We're almost ready to train the model. With the hyper-parameters we've chosen, training should go fairly quickly, converging to reasonably good results after around 10 epochs.

Besides reporting the loss, we can make some sense of our progress by examining nearest neighbors of a few test words in the embedding space. We'll compute cosine similarity between each of these words and the rest of the vocabulary, and show a few nearest neighbors. The semantic similarity between nearest neighbors should improve as the model trains.

In [0]:
num_nearest_neighbors = 5 # Number of closest neighbors to print
test_words = ["fred", "garden", "mice", "where", "afterwards"]

def print_nearest_neighbors(test_words):
    word_ids = np.array(vocab.words_to_ids(test_words))
    words_ = tf.constant(word_ids, dtype=tf.int32)

    # Normalize all embeddings to unit length for better comparisons.
    normalized_embeddings_ = V_ / tf.sqrt(tf.reduce_sum(tf.square(V_), 1, keepdims=True))

    # Get the normalized embeddings for the test words.
    words_normalized_embeddings_ = tf.nn.embedding_lookup(normalized_embeddings_, words_)
    
    # Cosine similarity is just a dot-product once embeddings have been normalized.
    # Remember that similarity between identical vectors is 1.
    similarity_ = tf.matmul(words_normalized_embeddings_, tf.transpose(normalized_embeddings_))
    
    # Execute the tf operations with a session.
    evaluated_sim_ = similarity_.eval(session=session)

    # Print the words with the largest similarities.
    print "Nearest neighbors of test words:"
    for i in xrange(len(test_words)):
        # The argsort() function returns the indices of a vector corresponding
        # to sorted order (from smallest to largest).
        nearest = (-evaluated_sim_[i, :]).argsort()[1:num_nearest_neighbors+1]
        print "  '{:s}' -> ".format(test_words[i]),
        for k in xrange(num_nearest_neighbors):
            closest_word = vocab.id_to_word[nearest[k]]
            print "'{:s}'".format(closest_word),
        print ""

In [0]:
# TRAINING PARAMETERS
num_epochs = 10 # one epoch = one pass through the training data
batch_size = 256
alpha = 1.0 # learning rate

print_every = 1000

In [0]:
np.random.seed(42) # we set this for reproducability
session = tf.Session()
session.run(init_)

t0 = time.time()
for epoch in xrange(1, num_epochs+1):
    t0_epoch = time.time()
    epoch_cost = 0.0
    total_batches = 0
    print ""
    for i, batch in enumerate(batch_generator(train_windows, batch_size)):
        if (i % print_every == 0):
            print "[epoch {:d}] seen {:,} minibatches".format(epoch, i)
        epoch_cost += train_batch(session, batch, alpha)
        total_batches = i + 1

    avg_cost = epoch_cost / total_batches
    print "[epoch {:d}] Completed {:d} minibatches in {:s}".format(epoch, i, pretty_timedelta(since=t0_epoch))
    print "[epoch {:d}] Average cost: {:.03f}".format(epoch, avg_cost)
    print_nearest_neighbors(test_words)

In [0]:
sample_test_words = ["bathroom", "lion", "cinema", "jessica"]
print_nearest_neighbors(sample_test_words)

## Scoring

We'll score our model the same as the n-gram model, by computing perplexity over the dev set.

Recall that perplexity is just the exponentiated average cross-entropy loss:

$$ \text{Perplexity} = \left( \prod_i \frac{1}{Q(x_i)} \right)^{1/N} = \left( \prod_i 2^{- \log_2 Q(x_i)} \right)^{1/N} = 2^{\left(\frac{1}{N} \sum_i -\log_2 Q(x_i)\right)} = 2^{\tilde{CE}(P,Q)}$$

In practice, TF uses the natural log, so the loss will be scaled by a factor of $\ln 2$, but the base cancels and the perplexity scores will be the same.

Note that below we use loss_, which is the cross-entropy loss with the full softmax.

In [0]:
def score_batch(session, batch):
    inputs, targets = cbow_slice_batch(batch, N)
    feed_dict = {ids_:inputs, y_:targets}
    return session.run(loss_, feed_dict=feed_dict)

def score_dataset(data):
    total_cost = 0.0
    total_batches = 0
    for batch in batch_generator(data, 1000):
        total_cost += score_batch(session, batch)
        total_batches += 1

    return total_cost / total_batches

In [0]:
print "Train set perplexity: %.03f" % np.exp(score_dataset(train_windows))
print "Test set perplexity: %.03f" % np.exp(score_dataset(test_windows))

## Predicting center words

Let's use our trained model to try predicting the middle word in some example sentences. We'll provide a context (two words to the left and right) and ask our model to predict the word in the middle. Note that we can use the `pred_max_` function to retrieve the word with the largest probability, or the `pred_random_` function to sample from the predicted distribution over words. The former is deterministic, while the latter will give a different result with each run.

While our model produces reasonable embeddings, it's not very good at predicting the target word.

In [0]:
sentences = ["mary went to the cinema",
             "brian is carrying a football"]

# This function feeds our sequence of words into the model and 
# runs it to get a guess of the center word. Feed the entire
# sentence in as it will automatically strip away the middle word
# before it enters the context words into the model.
def predict_target(session, seq):
    inputs, _ = cbow_slice_batch(np.array(seq).reshape([1, -1]), N)
    middle_word_id = session.run(pred_max_, feed_dict={ids_:inputs}).flatten()
    return middle_word_id[0]

for sentence in sentences:
    words = sentence.split()
    for w in words:
        if not w in vocab.wordset: raise KeyError("Out-of-vocabulary word \"{:s}\"".format(w))
    
    sentence_ids = vocab.words_to_ids(words)
    pred_id = predict_target(session, sentence_ids)
    print "Actual sentence:  \"{:s}\"".format(sentence)
    print "Sampled sentence: \"{:s} [{:s}] {:s}\"".format(" ".join(words[:N//2]),
                                                          vocab.id_to_word[pred_id],
                                                          " ".join(words[N//2+1:]))
    print

## Takeaways

We've demonstrated the CBOW model for producing word embeddings by predicting center words given context words. Whereas applying SVD captures maximum variance in the co-occurrence matrix, CBOW chooses embeddings that are suited to prediction. This means that CBOW can learn what aspects or dimensions are useful for a task, which turns out to be important. So why should we care about producing these embeddings? There are two big reasons.

* **Embeddings are the foundation for most Neural Network models of any language task.** Usually, the end goal is not the embeddings themselves. Instead, we are more likely to care about something like classification of documents. In this case, we'd create a network where the first layer is an embedding lookup and the final layer is a softmax over the possible classes. In between, there are a variety of possible network achitectures, but they all typically start with some kind of embeddings.

* **Pre-trained embeddings facilitate transfer learning.** Transfer learning is the transfer of knowledge about one task to help with another task. For example, consider a classification task with only a few thousand training examples. That's not enough to learn the parameters for word embeddings (for any reasonable sized vocabulary). Instead, we can initialize our model with embeddings trained from a large corpus of text. This strategy often gives dramatic improvements in results. It's a form of semi-supervised learning. We can estimate word embeddings from heaps of unlabeled text, and apply that learned knowledge -- representations of meaning for words -- to new tasks with a little labeled data.

## Experiments

(1) In our implementation, we estimated separate (untied) parameters for the $V$ and $U$ matrices, the 'input' and 'output' embeddings. Modify the code to only use a single set of parameters. How does this affect the performance (test perplexity)?

(2) Modify the CBOW code here to implement SkipGram. Rather than averaging the context words to predict the center word, SkipGram takes a center word as input and tries to predict the context words.